In [1]:
from decimal import Decimal
from typing import Literal

from jetblack_pnl.core import (
    add_trade,
    TradingPnl,
    IMatchedPool,
    IUnmatchedPool,
)
from jetblack_pnl.impl.simple import (
    MatchedPool,
    UnmatchedPool,
    Security,
    Trade
)

class TradeData:

    def __init__(self, trade_id: int, description: str) -> None:
        self.trade_id = trade_id
        self.description = description

    def __repr__(self) -> str:
        return f"[{self.trade_id}] {self.description}"
    
Side = Literal['buy', 'sell']

def trade(trade_id: int, side: Side, quantity: int, price: int) -> Trade:
    sign = 1 if side == 'buy' else -1
    return Trade(quantity * sign, price)

def display(pnl: TradingPnl[str], matched: IMatchedPool[None, None], unmatched: IUnmatchedPool[None, None]) -> None:
    avg_cost = -pnl.cost / pnl.quantity if pnl.quantity != 0 else 0
    text = f"pnl: {pnl}, avg_cost: {avg_cost}" + "\n"
    text += "matched:\n"
    if not matched.pool(None):
        text += "    None\n"
    else:
        for trades in matched.pool(None):
            text += f"    " + str(trades) + "\n"
    text += "unmatched:\n"
    if not unmatched.pool(None):
        text += "    None\n"
    else:
        for trade in unmatched.pool(None):
            text += f"    " + str(trade) + "\n"
    print(text)

In [2]:
sec = Security("AAPL", 1, False)
trade_id = 0
matched = MatchedPool()
unmatched = UnmatchedPool.Fifo()
pnl = TradingPnl[str](Decimal(0), Decimal(0), Decimal(0))
display(pnl, matched, unmatched)

pnl: position: 0, cost: 0, realized: 0, avg_cost: 0
matched:
    None
unmatched:
    None



In [3]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 6, 100), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 6, cost: -600, realized: 0, avg_cost: 100
matched:
    None
unmatched:
    self.quantity=Decimal('6') <self.trade=buy 6 @ 100>



In [4]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 6, 106), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 12, cost: -1236, realized: 0, avg_cost: 103
matched:
    None
unmatched:
    self.quantity=Decimal('6') <self.trade=buy 6 @ 100>
    self.quantity=Decimal('6') <self.trade=buy 6 @ 106>



In [5]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 6, 103), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 18, cost: -1854, realized: 0, avg_cost: 103
matched:
    None
unmatched:
    self.quantity=Decimal('6') <self.trade=buy 6 @ 100>
    self.quantity=Decimal('6') <self.trade=buy 6 @ 106>
    self.quantity=Decimal('6') <self.trade=buy 6 @ 103>



In [6]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'sell', 9, 105), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 9, cost: -936, realized: 27, avg_cost: 104
matched:
    (self.quantity=Decimal('6') <self.trade=buy 6 @ 100>, self.quantity=Decimal('-6') <self.trade=sell 9 @ 105>)
    (self.quantity=Decimal('3') <self.trade=buy 6 @ 106>, self.quantity=Decimal('-3') <self.trade=sell 9 @ 105>)
unmatched:
    self.quantity=Decimal('3') <self.trade=buy 6 @ 106>
    self.quantity=Decimal('6') <self.trade=buy 6 @ 103>



In [7]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'sell', 12, 107), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: -3, cost: 321, realized: 54, avg_cost: 107
matched:
    (self.quantity=Decimal('6') <self.trade=buy 6 @ 100>, self.quantity=Decimal('-6') <self.trade=sell 9 @ 105>)
    (self.quantity=Decimal('3') <self.trade=buy 6 @ 106>, self.quantity=Decimal('-3') <self.trade=sell 9 @ 105>)
    (self.quantity=Decimal('3') <self.trade=buy 6 @ 106>, self.quantity=Decimal('-3') <self.trade=sell 12 @ 107>)
    (self.quantity=Decimal('6') <self.trade=buy 6 @ 103>, self.quantity=Decimal('-6') <self.trade=sell 12 @ 107>)
unmatched:
    self.quantity=Decimal('-3') <self.trade=sell 12 @ 107>



In [8]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 3, 105), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 0, cost: 0, realized: 60, avg_cost: 0
matched:
    (self.quantity=Decimal('6') <self.trade=buy 6 @ 100>, self.quantity=Decimal('-6') <self.trade=sell 9 @ 105>)
    (self.quantity=Decimal('3') <self.trade=buy 6 @ 106>, self.quantity=Decimal('-3') <self.trade=sell 9 @ 105>)
    (self.quantity=Decimal('3') <self.trade=buy 6 @ 106>, self.quantity=Decimal('-3') <self.trade=sell 12 @ 107>)
    (self.quantity=Decimal('6') <self.trade=buy 6 @ 103>, self.quantity=Decimal('-6') <self.trade=sell 12 @ 107>)
    (self.quantity=Decimal('-3') <self.trade=sell 12 @ 107>, self.quantity=Decimal('3') <self.trade=buy 3 @ 105>)
unmatched:
    None

